In [ ]:
# Libraries

import random
import operator

import numpy as np
import matplotlib.pyplot as plt

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

In [ ]:
# Create individual minimizing 2 objectives (false positives & false negatives)

creator.create("FitnessMin", base.Fitness, weights=(-1.0,-1.0))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

In [1]:
# Setup of evolution environment (adding primitives, terminals, etc.)

random.seed(25)

pset = gp.PrimitiveSet("MAIN", arity=1)
pset.addPrimitive(np.add, arity=2)
pset.addPrimitive(np.subtract, arity=2)
pset.addPrimitive(np.multiply, arity=2)
pset.addPrimitive(np.negative, arity=1)

# Use the following example if you want strongly-typed GP

# pset = PrimitiveSetTyped("main", [bool, float], float)
# pset.addPrimitive(operator.xor, [bool, bool], bool)
# pset.addPrimitive(operator.mul, [float, float], float)
# pset.addTerminal(3.0, float) # Constants
# pset.addTerminal(1, bool) # Constants
# pset.addEphemeralConstant(lambda: random.randint(-10, 10), int) # Constants

# Here, use the example renameArgument function calls for every column in the titanic input data besides survived
# pset.renameArguments(ARG0='x')
# pset.renameArguments(ARG1='x')
# ...


toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

# Evaluation Function; this will have a return value in the form of (False Positive Rate, False Negative Rate) 
def evaluatePredictor(individual, points, pset):
    func = gp.compile(expr=individual, pset=pset)
    predictions = func(points)
    # Here, calculate the number of false positives & false negatives in predictions    
    falsePosRate = None
    truePosRate = None
    
    
    return (falsePosRate, truePosRate)


toolbox.register("evaluate", evaluatePredictor, points=, pset=pset) # After "points =" in this line, add the 
# processed titanic input data; Note points will have the same columns as the renameArguments calls above
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

# This controls the max depth our trees can build to; python limit is 91 levels; this sets it to 17
toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

SyntaxError: invalid syntax (1815319669.py, line 42)

In [ ]:
# Main Evolutionary Loop

gen = range(40)
avg_list = []
max_list = []
min_list = []

# Change 300 to any number of individuals we want to model
pop = toolbox.population(n=300)

# Evaluate the entire population
fitnesses = list(map(toolbox.evaluate, pop))
for ind, fit in zip(pop, fitnesses):
    ind.fitness.values = fit

# Begin the evolution
for g in gen:
    print("-- Generation %i --" % g)

    # Select the next generation individuals
    offspring = toolbox.select(pop, len(pop))
    # Clone the selected individuals
    offspring = list(map(toolbox.clone, offspring))

    # Apply crossover and mutation on the offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < 0.5:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < 0.2:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    # Replace population
    pop[:] = offspring

    # Gather all the fitnesses in one list and print the stats
    fits = [ind.fitness.values[0] for ind in pop]

    length = len(pop)
    mean = sum(fits) / length
    sum2 = sum(x*x for x in fits)
    std = abs(sum2 / length - mean**2)**0.5
    g_max = max(fits)
    g_min = min(fits)
        
    avg_list.append(mean)
    max_list.append(g_max)
    min_list.append(g_min)

    print("  Min %s" % g_min)
    print("  Max %s" % g_max)
    print("  Avg %s" % mean)
    print("  Std %s" % std)

print("-- End of (successful) evolution --")

best_ind = tools.selBest(pop, 1)[0]
print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))

# I think this loop from Lab 2 should work fine with some tweaks since we can't use eaMuPlusLambda